In [ ]:
import glob
import pandas as pd
import numpy as np
import json

In [ ]:
human_study_data_path = "../output/human_ai_study/"

participants_models_mappings = pd.read_excel("../python_vip/participants_models_mappings.xlsx")

In [ ]:
participants_models_mappings

In [ ]:
model_information = {}

with open("../python_vip/name_mappings.json") as f:
    model_information = json.load(f)

model_information

In [ ]:
tasks = {
    "session_1": {
        "pendulum": "pendulum", "rdk_alone": "rdk_alone_first", 
        "rdk_AI_assist": "rdk_alone_AI_assist_first", "rdk_hitl": "rdk_alone_hitl"
    }
}

Getting disagreement episodes from HITL task trials. 

get participant ids

sort them based on Number

group them based on the assistant AI

get disagreement episodes

compile and save them for each AI - done

train each AI 

In [ ]:
participant_data_paths = glob.glob(f"{human_study_data_path}*")
participant_ids = sorted([x.split('/')[-1] for x in participant_data_paths], 
                         key=lambda x: x.split('_')[-1])
participant_ids

In [ ]:
participants_models_mappings.head(21)

In [ ]:
model_ppt_groups = {}

for model in model_information['models']:
    model_name = model_information['models'][model]['name']
    model_ppt_groups[model_name] = {}
    model_ppt_groups[model_name]['ppts']  = participants_models_mappings[participants_models_mappings['model'] == model].dropna()['participant'].to_list()[:4]

model_ppt_groups

In [ ]:
for model in model_ppt_groups:
    print(model)
    dis_episodes_per_model = []

    for x in model_ppt_groups[model]['ppts']:
        dis_episodes_per_model.extend(glob.glob(f"{human_study_data_path}{x}/{tasks['session_1']['rdk_hitl']}/*/disagreement_episodes.txt"))
    
    print(dis_episodes_per_model)
    # model_ppt_groups[model]['paths'] = dis_episodes_per_model

    temp_data = []
    for pth in dis_episodes_per_model:
        print(pth)
        print("here")
        dis_episodes = np.loadtxt(pth, delimiter=',')
        
        if len(dis_episodes): 
            if dis_episodes.ndim != 2: dis_episodes = dis_episodes.reshape(-1, dis_episodes.size)
            temp_data.append(dis_episodes)
        else:
            print(len(dis_episodes))
    try:
        model_ppt_groups[model]['disagreement_data'] = np.concatenate(temp_data, axis=0)
    except Exception as e:
        print(temp_data)
        print(e)
        for i in range(len(temp_data)):
            print(i, temp_data[i].shape)
        print(temp_data[3].reshape(1, -1).shape)
        # print(temp_data)

    print(model_ppt_groups[model]['disagreement_data'].shape)

    np.savetxt(f"../data/human_ai_study/{model}_disagreement_episodes.txt", model_ppt_groups[model]['disagreement_data'],  delimiter=',')
# model_ppt_groups
    
    

In [ ]:
# model_ppt_groups

### retraining vanilla SAC model. 


In [ ]:

# sac_penalizeDeflection
import sys
sys.path.append("../")
import gymnasium as gym
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.dataset import Dataset, random_split

from stable_baselines3 import PPO, A2C, SAC, TD3, DDPG
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


from reinforcement.envs.pendulum_penalizeDeflection import PendulumEnv as penalizeDeflection

class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions

    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

def pretrain_agent(
    student,
    env,
    train_expert_dataset, test_expert_dataset,
    trace=[],
    batch_size=64,
    epochs=1000,
    scheduler_gamma=0.7,
    learning_rate=1.0,
    log_interval=100,
    no_cuda=False,
    seed=1,
    test_batch_size=64,
):
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(seed)
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    if isinstance(env.action_space, gym.spaces.Box):
        criterion = nn.MSELoss()
    else:
        criterion = nn.CrossEntropyLoss()

    # Extract initial policy
    model = student.policy.to(device)

    def train(model, device, train_loader, optimizer):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            if isinstance(env.action_space, gym.spaces.Box):
                # A2C/PPO policy outputs actions, values, log_prob
                # SAC/TD3 policy outputs actions only
                if isinstance(student, (A2C, PPO)):
                    action, _, _ = model(data)
                else:
                    # SAC/TD3:
                    action = model(data)
                action_prediction = action.double()
            else:
                # Retrieve the logits for A2C/PPO when using discrete actions
                dist = model.get_distribution(data)
                action_prediction = dist.distribution.logits
                target = target.long()

            loss = criterion(action_prediction, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % log_interval == 0:
                trace.append([epoch, loss.item()])
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                if isinstance(env.action_space, gym.spaces.Box):
                    # A2C/PPO policy outputs actions, values, log_prob
                    # SAC/TD3 policy outputs actions only
                    if isinstance(student, (A2C, PPO)):
                        action, _, _ = model(data)
                    else:
                        # SAC/TD3:
                        action = model(data)
                    action_prediction = action.double()
                else:
                    # Retrieve the logits for A2C/PPO when using discrete actions
                    dist = model.get_distribution(data)
                    action_prediction = dist.distribution.logits
                    target = target.long()

                test_loss = criterion(action_prediction, target)
        test_loss /= len(test_loader.dataset)
        print(f"Test set: Average loss: {test_loss:.4f}")

    # Here, we use PyTorch `DataLoader` to our load previously created `ExpertDataset` for training
    # and testing
    train_loader = torch.utils.data.DataLoader(
        dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    test_loader = torch.utils.data.DataLoader(
        dataset=test_expert_dataset,
        batch_size=test_batch_size,
        shuffle=True,
        **kwargs,
    )

    # Define an Optimizer and a learning rate schedule.
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

    # Now we are finally ready to train the policy model.
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer)
        test(model, device, test_loader)
        scheduler.step()

    # Implant the trained policy network back into the RL student agent
    student.policy = model

    return student



In [ ]:

model_name = "sac_penalizeDeflection"
model_id = "A"

model_dis_episodes = model_ppt_groups[model_name]['disagreement_data']

features, model_actions, human_actions = model_dis_episodes[:, :-2], model_dis_episodes[:, -2], model_dis_episodes[:, -1]

targets = model_actions * -1
targets = targets.reshape(-1, 1)

expert_observations = features
expert_actions = targets

train_config = {
    "model_type": "sac"
}

env = penalizeDeflection(render_mode='human')

n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


student_model = None

if train_config['model_type'] == 'ddpg':
    student_model = DDPG.load(model_information["models"][model_id]["path"])
else:
    student_model = SAC.load(model_information["models"][model_id]["path"])

# env = student_model.get_env()

train_config = {
    "model_type": "sac"
}

env = penalizeDeflection(render_mode='human')

n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


student_model = None



# env = student_model.get_env()
print(env)
print(student_model)

expert_dataset = ExpertDataSet(expert_observations, expert_actions)

train_size = int(0.9 * len(expert_dataset))

test_size = len(expert_dataset) - train_size

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

len(train_expert_dataset), len(test_expert_dataset)



# epochs=100

res = []



for epochs, lr in [(100, 1e-3), (100, 1e-4), (100, 1e-5), (200, 1e-3), (200, 1e-4), (200, 1e-5),]:
    trace = []
    env = penalizeDeflection(render_mode='human')

    if train_config['model_type'] == 'ddpg':
        student_model = DDPG.load(model_information["models"][model_id]["path"])
    else:
        student_model = SAC.load(model_information["models"][model_id]["path"])
    # env.
    updated_model = pretrain_agent(
        student_model,
        env,
        train_expert_dataset, test_expert_dataset,
        trace=trace,
        epochs=epochs,
        scheduler_gamma=0.7,
        learning_rate=lr,
        log_interval=100,
        no_cuda=False,
        seed=1,
        batch_size=64,
        test_batch_size=10,
    )

    mean_reward, std_reward = evaluate_policy(updated_model, env, n_eval_episodes=5)
    print(f"Mean reward = {mean_reward} +/- {std_reward}")

    env.close()

    res.append([mean_reward, std_reward, epochs, lr])


res
    # break

if train_config['model_type'] == 'ddpg':
    student_model = DDPG.load(model_information["models"][model_id]["path"])
else:
    student_model = SAC.load(model_information["models"][model_id]["path"])

env = penalizeDeflection(render_mode='human')

mean_reward, std_reward = evaluate_policy(student_model, env, n_eval_episodes=5)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

env.close()

res.append([mean_reward, std_reward, None, None])

res = pd.DataFrame(res, columns=["mean_reward", "std_reward", "epochs", "lr"])
res = res.sort_values(["mean_reward", "std_reward"], ascending=False)

epochs, lr = None, None
if res.iloc[0].epochs != None:
    epochs, lr = int(res.iloc[0].epochs), res.iloc[0].lr
else:
    epochs, lr = int(res.iloc[1].epochs), res.iloc[1].lr

updated_model = pretrain_agent(
        student_model,
        env,
        train_expert_dataset, test_expert_dataset,
        trace=trace,
        epochs=epochs,
        scheduler_gamma=0.7,
        learning_rate=lr,
        log_interval=100,
        no_cuda=False,
        seed=1,
        batch_size=64,
        test_batch_size=10,
    )

env = penalizeDeflection(render_mode='human')

mean_reward, std_reward = evaluate_policy(updated_model, env, n_eval_episodes=5)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

updated_model.save(f"../working_models/assistants/{model_name}_alpha_study_batch_1")



### retraining vanilla DDPG model. 

In [ ]:


# ddpg_penalizeDeflection

model_name = "ddpg_penalizeDeflection"
model_id = "C"

model_dis_episodes = model_ppt_groups[model_name]['disagreement_data']

features, model_actions, human_actions = model_dis_episodes[:, :-2], model_dis_episodes[:, -2], model_dis_episodes[:, -1]

targets = model_actions * -1
targets = targets.reshape(-1, 1)

expert_observations = features
expert_actions = targets

train_config = {
    "model_type": "ddpg"
}

env = penalizeDeflection(render_mode='human')

n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


student_model = None

if train_config['model_type'] == 'ddpg':
    student_model = DDPG.load(model_information["models"][model_id]["path"])
else:
    student_model = SAC.load(model_information["models"][model_id]["path"])

# env = student_model.get_env()

train_config = {
    "model_type": "ddpg"
}

env = penalizeDeflection(render_mode='human')

n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


student_model = None



# env = student_model.get_env()
print(env)
print(student_model)

expert_dataset = ExpertDataSet(expert_observations, expert_actions)

train_size = int(0.9 * len(expert_dataset))

test_size = len(expert_dataset) - train_size

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

len(train_expert_dataset), len(test_expert_dataset)



# epochs=100

res = []



for epochs, lr in [(100, 1e-3), (100, 1e-4), (100, 1e-5), (200, 1e-3), (200, 1e-4), (200, 1e-5),]:
    trace = []
    env = penalizeDeflection(render_mode='human')

    if train_config['model_type'] == 'ddpg':
        student_model = DDPG.load(model_information["models"][model_id]["path"])
    else:
        student_model = SAC.load(model_information["models"][model_id]["path"])
    # env.
    updated_model = pretrain_agent(
        student_model,
        env,
        train_expert_dataset, test_expert_dataset,
        trace=trace,
        epochs=epochs,
        scheduler_gamma=0.7,
        learning_rate=lr,
        log_interval=100,
        no_cuda=False,
        seed=1,
        batch_size=64,
        test_batch_size=10,
    )

    mean_reward, std_reward = evaluate_policy(updated_model, env, n_eval_episodes=5)
    print(f"Mean reward = {mean_reward} +/- {std_reward}")

    env.close()

    res.append([mean_reward, std_reward, epochs, lr])


res
    # break

if train_config['model_type'] == 'ddpg':
    student_model = DDPG.load(model_information["models"][model_id]["path"])
else:
    student_model = SAC.load(model_information["models"][model_id]["path"])

env = penalizeDeflection(render_mode='human')

mean_reward, std_reward = evaluate_policy(student_model, env, n_eval_episodes=5)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

env.close()

res.append([mean_reward, std_reward, None, None])

res = pd.DataFrame(res, columns=["mean_reward", "std_reward", "epochs", "lr"])
res = res.sort_values(["mean_reward", "std_reward"], ascending=False)
print(res)
epochs, lr = None, None
if not np.isnan(res.iloc[0].epochs):
    epochs, lr = int(res.iloc[0].epochs), res.iloc[0].lr
else:
    epochs, lr = int(res.iloc[1].epochs), res.iloc[1].lr

updated_model = pretrain_agent(
        student_model,
        env,
        train_expert_dataset, test_expert_dataset,
        trace=trace,
        epochs=epochs,
        scheduler_gamma=0.7,
        learning_rate=lr,
        log_interval=100,
        no_cuda=False,
        seed=1,
        batch_size=64,
        test_batch_size=10,
    )

env = penalizeDeflection(render_mode='human')

mean_reward, std_reward = evaluate_policy(updated_model, env, n_eval_episodes=5)
print(f"Mean reward = {mean_reward} +/- {std_reward}")

updated_model.save(f"../working_models/assistants/{model_name}_alpha_study_batch_1")



### retraining MLP all prof assistant

In [ ]:
import sys
sys.path.append('../')
from deep_learning.utils.utils import *

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


import torch
import pandas as pd

from python_vip.vip import load_model, print_model_summary



from torch.utils.data import DataLoader

In [ ]:
class RetrainDataset(Dataset):
    def __init__(self, X, Y) -> None:
        super().__init__()
        if not isinstance(X, torch.Tensor):
            self.X = torch.from_numpy(X)
            self.Y = torch.from_numpy(Y)
        else:
            self.X = X
            self.Y = Y

    def __len__(self,):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx].float(), self.Y[idx].float()

In [ ]:
data_path = '../data/'

balancing_data_path = f"{data_path}data_all_radians.csv"
proficiencies_data_path = f"{data_path}overall_proficiency.csv"
print("Data file used: ", balancing_data_path)
print("Proficiency file used: ", proficiencies_data_path)

orig_data = pd.read_csv(balancing_data_path)
orig_data['joystickX'] = -1 * orig_data['joystickX']

proficiencies = pd.read_csv(proficiencies_data_path)
ppts, ppt_data = filter_data_based_proficiency(orig_data, proficiencies, proficiency=["Good"])
train_data, test_data = train_test_split_trials(ppt_data, ppts, train=0.9)




In [ ]:
train_ml_dataset = MarsDataset(train_data, past_window_size=0, future_time=1, lstm_format=False)

test_ml_dataset = MarsDataset(test_data, past_window_size=0,future_time=1, lstm_format=False, Xmeans=train_ml_dataset.Xmeans, Xstds=train_ml_dataset.Xstds, Tmeans=train_ml_dataset.Tmeans, Tstds=train_ml_dataset.Tstds)

test_ppt_trial = MarsDataset(orig_data[orig_data.peopleTrialKey == '2_ms_P2/20_600back_Block5_trial_020.csv'], past_window_size=0, future_time=1, lstm_format=False, Xmeans=train_ml_dataset.Xmeans, Xstds=train_ml_dataset.Xstds, Tmeans=train_ml_dataset.Tmeans, Tstds=train_ml_dataset.Tstds)


In [ ]:
model_name = "assistant_mars_mlp_all_prof"
model_id = "D"

config = model_information["models"][model_id]

epochs_lrs = [(50, 1e-3), (50, 1e-4), (50, 1e-5), (100, 1e-3), (100, 1e-4), (100, 1e-5),]

model_dis_episodes = model_ppt_groups[model_name]['disagreement_data']

features, model_actions, human_actions = model_dis_episodes[:, :-2], model_dis_episodes[:, -2], model_dis_episodes[:, -1]

targets = -1 * model_actions.reshape(-1, 1)

features = torch.from_numpy(features).float()
targets = torch.from_numpy(targets).float()

trainX_hitl, testX_hitl, trainY_hitl, testY_hitl = train_test_split(features, targets, test_size=0.1, random_state=42)

train_loader, test_loader = None, None

trainX = (trainX_hitl - train_ml_dataset.Xmeans) / train_ml_dataset.Xstds
trainY = (trainY_hitl - train_ml_dataset.Tmeans) / train_ml_dataset.Tstds

testX = (testX_hitl - train_ml_dataset.Xmeans) / train_ml_dataset.Xstds
testY = (testY_hitl - train_ml_dataset.Tmeans) / train_ml_dataset.Tstds

if config['type'] in {'lstm', 'gru', 'rnn'}:
    trainX = trainX.reshape(-1, config['past_window_size']+1, 3)
    testX = testX.reshape(-1, config['past_window_size']+1, 3)

train_set = RetrainDataset(trainX, trainY)
test_set = RetrainDataset(testX, testY)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=10)
test_loader = DataLoader(test_set, batch_size=16, num_workers=10)


results = []


for epochs, lr in epochs_lrs:
    model = load_model(config["path"], config['type'])
    model.lr = lr
    name = f"{config['name']}_hitl_retrain_epochs_{epochs}_lr_{lr}"

    checkpoint_callback = ModelCheckpoint(
                            monitor='val_MAE',
                            dirpath=f"./output/{name}/",
                            filename='best_checkpoint',
                            save_top_k=1,
                            mode='min',
                        )

    early_stopping = EarlyStopping(
        monitor='val_MAE',
        min_delta=0.00001,
        patience= 5,
        mode='min'
    )

    logger = TensorBoardLogger('lightning_logs', name=f"{name}")


    trainer = pl.Trainer(
        max_epochs=epochs,
        precision=32,
        accelerator="auto",
        num_sanity_val_steps=0,
        check_val_every_n_epoch=1,
        callbacks=[checkpoint_callback, early_stopping],
        logger=logger,
        devices=[0]
        # strategy='ddp'
    )

    # trainer.
    trainer.fit(model, train_loader, test_loader) 

    testX = testX_hitl
    testY = testY_hitl

    # human_actions = disagreement_data[:, -1]
    if config['type'] in {'lstm', 'gru', 'rnn'}:
        testX = testX.reshape(-1, config['past_window_size']+1, 3)
        
    pred_joys = model.use(testX)
    # dl_seconds = np.float32(test_ppt_dataset.metas[:,0])
    actual_joys = testY


    mae_test_hitl = mean_absolute_error(actual_joys, pred_joys),
    max_err_test_hitl = max_error(actual_joys, pred_joys)
    rmse_test_hitl = mean_squared_error(actual_joys, pred_joys, squared=False)

    features = test_ppt_trial.features_physics.float()
                # print(features.shape)
    Xs = features.reshape(-1,config['past_window_size']+1, 3) if config['type'] != 'mlp' else features
    pred_joys = model.use(Xs)
    # dl_seconds = np.float32(test_ppt_dataset.metas[:,0])
    actual_joys = test_ppt_trial.targets.numpy()


    mae_test_ppt = mean_absolute_error(actual_joys, pred_joys),
    max_err_test_ppt = max_error(actual_joys, pred_joys)
    rmse_test_ppt = mean_squared_error(actual_joys, pred_joys, squared=False)

    results.append([epochs, lr, mae_test_ppt[0], rmse_test_ppt, max_err_test_ppt, mae_test_hitl[0], rmse_test_hitl, max_err_test_hitl])



res = pd.DataFrame(results, columns=['epochs', 'lr', 'mae_trial', 'rmse_trial', 'max_err_trial', 'mae_hitl', 'rmse_hitl', 'max_err_hitl'])

In [ ]:
res

In [ ]:
train_ml_dataset = MarsDataset(train_data, past_window_size=10, future_time=1, lstm_format=True)

test_ml_dataset = MarsDataset(test_data, past_window_size=10,future_time=1, lstm_format=True, Xmeans=train_ml_dataset.Xmeans, Xstds=train_ml_dataset.Xstds, Tmeans=train_ml_dataset.Tmeans, Tstds=train_ml_dataset.Tstds)

test_ppt_trial = MarsDataset(orig_data[orig_data.peopleTrialKey == '2_ms_P2/20_600back_Block5_trial_020.csv'], past_window_size=10, future_time=1, lstm_format=True, Xmeans=train_ml_dataset.Xmeans, Xstds=train_ml_dataset.Xstds, Tmeans=train_ml_dataset.Tmeans, Tstds=train_ml_dataset.Tstds)

In [ ]:
model_name = "assistant_mars_lstm_good_small_window"
model_id = "E"

config = model_information["models"][model_id]

epochs_lrs = [(50, 1e-3), (50, 1e-4), (50, 1e-5), (100, 1e-3), (100, 1e-4), (100, 1e-5),]

model_dis_episodes = model_ppt_groups[model_name]['disagreement_data']

features, model_actions, human_actions = model_dis_episodes[:, :-2], model_dis_episodes[:, -2], model_dis_episodes[:, -1]

targets = -1 * model_actions.reshape(-1, 1)

features = torch.from_numpy(features).float()
targets = torch.from_numpy(targets).float()

trainX_hitl, testX_hitl, trainY_hitl, testY_hitl = train_test_split(features, targets, test_size=0.1, random_state=42)

train_loader, test_loader = None, None
print(trainX_hitl.shape, train_ml_dataset.Xmeans.shape, train_ml_dataset.Xstds.shape)
trainX = (trainX_hitl - train_ml_dataset.Xmeans) / train_ml_dataset.Xstds
trainY = (trainY_hitl - train_ml_dataset.Tmeans) / train_ml_dataset.Tstds

testX = (testX_hitl - train_ml_dataset.Xmeans) / train_ml_dataset.Xstds
testY = (testY_hitl - train_ml_dataset.Tmeans) / train_ml_dataset.Tstds

if config['type'] in {'lstm', 'gru', 'rnn'}:
    trainX = trainX.reshape(-1, train_ml_dataset.Xstds.shape[0]//3, 3)
    testX = testX.reshape(-1, train_ml_dataset.Xstds.shape[0]//3, 3)

train_set = RetrainDataset(trainX, trainY)
test_set = RetrainDataset(testX, testY)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=10)
test_loader = DataLoader(test_set, batch_size=16, num_workers=10)


results = []


for epochs, lr in epochs_lrs:
    model = load_model(config["path"], config['type'])
    model.lr = lr
    name = f"{config['name']}_hitl_retrain_epochs_{epochs}_lr_{lr}"

    checkpoint_callback = ModelCheckpoint(
                            monitor='val_MAE',
                            dirpath=f"./output/{name}/",
                            filename='best_checkpoint',
                            save_top_k=1,
                            mode='min',
                        )

    early_stopping = EarlyStopping(
        monitor='val_MAE',
        min_delta=0.00001,
        patience= 5,
        mode='min'
    )

    logger = TensorBoardLogger('lightning_logs', name=f"{name}")


    trainer = pl.Trainer(
        max_epochs=epochs,
        precision=32,
        accelerator="auto",
        num_sanity_val_steps=0,
        check_val_every_n_epoch=1,
        callbacks=[checkpoint_callback, early_stopping],
        logger=logger,
        devices=[0]
        # strategy='ddp'
    )

    # trainer.
    trainer.fit(model, train_loader, test_loader) 

    testX = testX_hitl
    testY = testY_hitl

    # human_actions = disagreement_data[:, -1]
    if config['type'] in {'lstm', 'gru', 'rnn'}:
        testX = testX.reshape(-1, train_ml_dataset.Xstds.shape[0]//3, 3)
        
    pred_joys = model.use(testX)
    # dl_seconds = np.float32(test_ppt_dataset.metas[:,0])
    actual_joys = testY


    mae_test_hitl = mean_absolute_error(actual_joys, pred_joys),
    max_err_test_hitl = max_error(actual_joys, pred_joys)
    rmse_test_hitl = mean_squared_error(actual_joys, pred_joys, squared=False)

    features = test_ppt_trial.features_physics.float()
                # print(features.shape)
    Xs = features.reshape(-1,train_ml_dataset.Xstds.shape[0]//3, 3) if config['type'] != 'mlp' else features
    pred_joys = model.use(Xs)
    # dl_seconds = np.float32(test_ppt_dataset.metas[:,0])
    actual_joys = test_ppt_trial.targets.numpy()


    mae_test_ppt = mean_absolute_error(actual_joys, pred_joys),
    max_err_test_ppt = max_error(actual_joys, pred_joys)
    rmse_test_ppt = mean_squared_error(actual_joys, pred_joys, squared=False)

    results.append([epochs, lr, mae_test_ppt[0], rmse_test_ppt, max_err_test_ppt, mae_test_hitl[0], rmse_test_hitl, max_err_test_hitl])



res = pd.DataFrame(results, columns=['epochs', 'lr', 'mae_trial', 'rmse_trial', 'max_err_trial', 'mae_hitl', 'rmse_hitl', 'max_err_hitl'])

In [ ]:
res

### retraining AIRL SAC model

In [ ]:

# sac_penalizeDeflection
import sys
sys.path.append("../")
import gym
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.dataset import Dataset, random_split

from stable_baselines3 import PPO, A2C, SAC, TD3, DDPG
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from imitation.data.types import Transitions
from stable_baselines3.common.env_util import make_vec_env
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm

from reinforcement.envs.pendulumV21 import PendulumEnv


model_name = "sac_airl_300k"
model_id = "B"

model_dis_episodes = model_ppt_groups[model_name]['disagreement_data']

features, model_actions, human_actions = model_dis_episodes[:, :-2], model_dis_episodes[:, -2], model_dis_episodes[:, -1]

targets = model_actions * -1
targets = targets.reshape(-1, 1)


env = PendulumEnv()
# curr_observations = []
next_observations = []
infos = []
dones = []

for i in range(len(features)):
    env.reset()
    obs = features[i]
    obs = [np.arctan2(obs[1], obs[0]), obs[2]]
    # curr_observations.append(obs)
    action = targets[i]
    env.state = obs
    next_obs, _, terminated, info = env.step(action)
    # print(next_obs)
    next_observations.append(next_obs)
    infos.append(info)

    dones.append(terminated)


env.close()

print(np.array(features).shape, np.array(targets).shape, np.array(infos).shape, np.array(next_observations).shape, np.array(dones).shape)

demos = Transitions(
    np.array(features, dtype=np.float64), np.array(targets, dtype=np.float64), np.array(infos), np.array(next_observations, dtype=np.float64), np.array(dones)
)



n_actions = env.action_space.shape[-1]
print(env.action_space)
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))


res = []

for epochs, lr in [(100, 1e-3), (100, 1e-4), (100, 1e-5), (200, 1e-3), (200, 1e-4), (200, 1e-5),]:
    env = PendulumEnv()
    venv = make_vec_env(PendulumEnv, n_envs=8)
    
    
    learner = SAC.load(model_information["models"][model_id]["path"])
    learner.env = venv
    learner.action_noise = action_noise
    learner.learning_rate = lr

    reward_net = BasicShapedRewardNet(
        venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
    )
    # env.

    airl_trainer = AIRL(
        demonstrations=demos,
        demo_batch_size=64,
        gen_replay_buffer_capacity=2048,
        n_disc_updates_per_round=4,
        venv=venv,
        gen_algo=learner,
        reward_net=reward_net,
        allow_variable_horizon=True
    )

    airl_trainer.train(epochs)
    
    mean_reward, std_reward = evaluate_policy(learner, env, n_eval_episodes=5)
    print(f"Mean reward = {mean_reward} +/- {std_reward}")

    venv.close()
    env.close()

    res.append([mean_reward, std_reward, epochs, lr, learner])


res


Looked at the highest mean reward and saved the model manually.

In [ ]:
res[0][-1].save("../working_models/assistants/sac_airl_300k_alpha_study_batch_1.zip")